In [31]:
from pymongo import MongoClient
from datetime import datetime, timezone
import pandas as pd

# Kết nối đến cơ sở dữ liệu MongoDB
client = MongoClient('localhost', 27017)
db = client['data_mining']  
collection = db['redditdata']  

In [32]:
for record in collection.find():
    created_utc = datetime.fromtimestamp(record['created_utc'], timezone.utc)
    collection.update_one({'_id': record['_id']}, {'$set': {'created_utc': created_utc}})

TypeError: 'datetime.datetime' object cannot be interpreted as an integer

In [33]:
pipeline = [
    {
        '$group': {
            '_id': {'$dateToString': {'format': '%Y-%m-%d', 'date': '$created_utc'}},
            'count': {'$sum': 1}
        }
    },
    {'$sort': {'_id': 1}}
]

# Thực hiện aggregation
results = collection.aggregate(pipeline)

# In ra kết quả thống kê
for result in results:
    print(f"Ngày: {result['_id']}, Số lượng bài đăng: {result['count']}")

Ngày: 2024-05-10, Số lượng bài đăng: 824
Ngày: 2024-05-11, Số lượng bài đăng: 10483


In [34]:
pipeline = [
    {"$group": {"_id": {"title": "$title", "subreddit": "$subreddit", "selftext": "$selftext", "num_comments": "$num_comments", "created_utc": "$created_utc"}, "count": {"$sum": 1}}},
    {"$match": {"count": {"$gt": 1}}}
]

# Lấy danh sách các bản ghi trùng nhau
duplicate_records = list(collection.aggregate(pipeline))

# Số lượng bản ghi trùng nhau
num_duplicates = len(duplicate_records)

# Loại bỏ các bản ghi trùng nhau và in ra thông báo
if num_duplicates > 0:
    for record in duplicate_records:
        title = record["_id"]["title"]
        subreddit = record["_id"]["subreddit"]
        selftext = record["_id"]["selftext"]
        num_comments = record["_id"]["num_comments"]
        created_utc = record["_id"]["created_utc"]
        duplicate_documents = collection.find({"title": title, "subreddit": subreddit, "selftext": selftext, "num_comments": num_comments, "created_utc": created_utc})
        # Loại bỏ tất cả các bản ghi trùng nhau, chỉ giữ lại một bản ghi
        for idx, doc in enumerate(duplicate_documents):
            if idx > 0:
                collection.delete_one({"_id": doc["_id"]})
    print(f"Đã loại bỏ {num_duplicates} bản ghi trùng nhau.")
else:
    print("Không có bản ghi trùng nhau cần loại bỏ.")

Đã loại bỏ 1689 bản ghi trùng nhau.


In [36]:
for record in collection.find():
    
    created_datetime = record['created_utc']
    
    
    hour_frame = created_datetime.hour // 6  
    day_of_week = created_datetime.weekday()  
    
    update_query = {'$set': {}}
    update_query['$set']['hour_frame'] = hour_frame
    update_query['$set']['day_of_week'] = day_of_week
    
    collection.update_one({'_id': record['_id']}, update_query)

In [37]:
for record in collection.find():
    day_of_week = record['day_of_week']
    
    days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    day_labels = ['is' + day for day in days_of_week]
    
    update_query = {'$set': {}}
    for i, day_label in enumerate(day_labels):
        update_query['$set'][day_label] = int(i == day_of_week)
    
    collection.update_one({'_id': record['_id']}, update_query)

In [38]:
for record in collection.find():
    hour_frame = record['hour_frame']
    
    hour_labels = ['isNight', 'isMorning', 'isAfternoon', 'isEvening']
    
    update_query = {'$set': {}}
    for i, hour_label in enumerate(hour_labels):
        update_query['$set'][hour_label] = int(i == hour_frame)
    
    collection.update_one({'_id': record['_id']}, update_query)

In [39]:
for record in collection.find():
    record_id = record['_id']
    
    update_query = {'$unset': {'day_of_week': '', 'hour_frame': ''}}
    
    collection.update_one({'_id': record_id}, update_query)

In [40]:
for record in collection.find():
    title_length = len(record['title'])
    selftext_length = len(record['selftext'])
    
    record_id = record['_id']
    
    update_query = {'$set': {'title_length': title_length, 'selftext_length': selftext_length}}
    
    collection.update_one({'_id': record_id}, update_query)

In [41]:
update_query = {'$set': {'domain': 'noURL'}}
collection.update_many({'domain': {'$regex': '^self\.'}}, update_query)

print(f"{collection.count_documents({'domain': 'noURL'})} bản ghi đã được cập nhật.")

4242 bản ghi đã được cập nhật.


In [42]:
import re
reddit_records = collection.find({'domain': 'reddit.com'})

domain_pattern = re.compile(r'https://(?:www\.)?([^/]+)')

for record in reddit_records:
    url = record['url']
    
    match = domain_pattern.match(url)
    if match:
        domain = match.group(1)
    else:
        domain = None
    
    collection.update_one({'_id': record['_id']}, {'$set': {'domain': domain}})

In [43]:
pipeline = [
    {'$group': {'_id': '$domain', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}}
]

domain_stats = list(collection.aggregate(pipeline))

print(len(domain_stats))

for domain_stat in domain_stats:
    print(f"Domain: {domain_stat['_id']}, Số lượng: {domain_stat['count']}")

236
Domain: noURL, Số lượng: 4242
Domain: x.com, Số lượng: 269
Domain: twitter.com, Số lượng: 213
Domain: youtube.com, Số lượng: 120
Domain: i.imgur.com, Số lượng: 101
Domain: youtu.be, Số lượng: 81
Domain: cnn.com, Số lượng: 81
Domain: streamable.com, Số lượng: 73
Domain: theguardian.com, Số lượng: 64
Domain: newsweek.com, Số lượng: 60
Domain: washingtonpost.com, Số lượng: 49
Domain: propublica.org, Số lượng: 48
Domain: salon.com, Số lượng: 46
Domain: rawstory.com, Số lượng: 43
Domain: nytimes.com, Số lượng: 42
Domain: en.wikipedia.org, Số lượng: 41
Domain: reuters.com, Số lượng: 40
Domain: bbc.co.uk, Số lượng: 40
Domain: thehill.com, Số lượng: 40
Domain: telegraph.co.uk, Số lượng: 37
Domain: nbcnews.com, Số lượng: 36
Domain: bbc.com, Số lượng: 35
Domain: businessinsider.com, Số lượng: 35
Domain: clips.twitch.tv, Số lượng: 34
Domain: pcgamer.com, Số lượng: 32
Domain: instagram.com, Số lượng: 32
Domain: jalopnik.com, Số lượng: 31
Domain: mil.in.ua, Số lượng: 31
Domain: pravda.com.ua, S

In [44]:
equivalent_sources = {
    'youtu.be': 'youtube.com',
    'i.imgur.com': 'imgur.com',
    'bbc.co.uk': 'bbc.com',
    'finance.yahoo.com': 'yahoo.com',
    'ca.style.yahoo.com': 'yahoo.com',
    'sports.yahoo.com': 'yahoo.com',
    'ca.news.yahoo.com': 'yahoo.com'
}

for source, equivalent_source in equivalent_sources.items():
    collection.update_many({'domain': source}, {'$set': {'domain': equivalent_source}})

# Thống kê số lượng bài đăng của mỗi domain
pipeline = [
    {'$group': {'_id': '$domain', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}}
]

domain_stats = list(collection.aggregate(pipeline))

# In ra kết quả
for domain_stat in domain_stats:
    print(f"Domain: {domain_stat['_id']}, Số lượng: {domain_stat['count']}")

Domain: noURL, Số lượng: 4242
Domain: x.com, Số lượng: 269
Domain: twitter.com, Số lượng: 213
Domain: youtube.com, Số lượng: 201
Domain: imgur.com, Số lượng: 106
Domain: cnn.com, Số lượng: 81
Domain: bbc.com, Số lượng: 75
Domain: streamable.com, Số lượng: 73
Domain: theguardian.com, Số lượng: 64
Domain: newsweek.com, Số lượng: 60
Domain: washingtonpost.com, Số lượng: 49
Domain: propublica.org, Số lượng: 48
Domain: salon.com, Số lượng: 46
Domain: rawstory.com, Số lượng: 43
Domain: nytimes.com, Số lượng: 42
Domain: en.wikipedia.org, Số lượng: 41
Domain: reuters.com, Số lượng: 40
Domain: thehill.com, Số lượng: 40
Domain: telegraph.co.uk, Số lượng: 37
Domain: nbcnews.com, Số lượng: 36
Domain: businessinsider.com, Số lượng: 35
Domain: clips.twitch.tv, Số lượng: 34
Domain: pcgamer.com, Số lượng: 32
Domain: instagram.com, Số lượng: 32
Domain: jalopnik.com, Số lượng: 31
Domain: mil.in.ua, Số lượng: 31
Domain: pravda.com.ua, Số lượng: 29
Domain: futurism.com, Số lượng: 29
Domain: jpost.com, Số 

In [45]:
pipeline = [
    {"$group": {"_id": "$domain", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}  
]

result = list(collection.aggregate(pipeline))

top_20_domains = [item['_id'] for item in result[:20]]
other_domains = [item['_id'] for item in result[20:]]

for record in collection.find():
    new_columns = {}
    for domain in top_20_domains:
        new_columns[domain.split('.')[0]] = 1 if record['domain'] == domain else 0
    new_columns['Other'] = 1 if record['domain'] in other_domains else 0
    
    collection.update_one({'_id': record['_id']}, {'$set': new_columns})

In [46]:
for record in collection.find():
    posttext = record.get('title', '') + ' ' + record.get('selftext', '')
    
    collection.update_one({'_id': record['_id']}, {'$set': {'posttext': posttext}})

In [47]:
import nltk
from collections import Counter

# Tải stopwords và tokenizer từ NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Lấy tất cả các bản ghi từ cơ sở dữ liệu
records = collection.find()

# Khởi tạo Counter để đếm số lần xuất hiện của các từ
word_counter = Counter()

# Duyệt qua tất cả các bản ghi và thêm từ vào Counter
for record in records:
    # Kết hợp title và selftext thành một văn bản
    text = record['title'] + ' ' + record['selftext']
    
    # Tokenize văn bản
    tokens = nltk.word_tokenize(text)
    
    # Loại bỏ stopwords và chuyển các từ về dạng chữ thường
    stop_words = set(nltk.corpus.stopwords.words('english'))
    words = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
    
    # Đếm số lần xuất hiện của các từ và cập nhật Counter
    word_counter.update(words)

# Lấy 200 bag of words phổ biến nhất
top_200_words = word_counter.most_common(200)

# Tạo danh sách tên trường và giá trị của chúng
field_names = [word for word, _ in top_200_words]
field_values = {word: count for word, count in top_200_words}

# Cập nhật mỗi bản ghi trong collection với các trường mới
for record in collection.find():
    # Kết hợp title và selftext thành một văn bản
    text = record['title'] + ' ' + record['selftext']
    
    # Tokenize văn bản
    tokens = nltk.word_tokenize(text)
    
    # Loại bỏ stopwords và chuyển các từ về dạng chữ thường
    stop_words = set(nltk.corpus.stopwords.words('english'))
    words = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
    
    # Đếm số lần xuất hiện của các từ
    word_count = {word: words.count(word) for word in field_names}
    
    # Cập nhật bản ghi với các trường mới
    collection.update_one({'_id': record['_id']}, {'$set': word_count})


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
collection.update_many({}, {"$unset": {"created_utc": ""}})

collection.update_many({}, {"$unset": {"title": "", "selftext": "", "subreddit": "", "domain": "", "url": "", "base_time": "", "posttext": ""}})

UpdateResult({'n': 7633, 'nModified': 7633, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)